In [15]:
import json
from bs4 import BeautifulSoup
import duckdb as db
import re
import numpy as np
import pandas as pd
from datetime import datetime
import locale
import requests
import os
from tqdm import tqdm

In [2]:
# Connect to the DuckDB database
con = db.connect(database='plenary_minutes.duckdb', read_only=False)

#Set locale for date parsing
locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')

'de_DE.UTF-8'

In [45]:
def create_tables():
    """
    Create tables in the DuckDB database for storing plenary minutes data.
    """
    con.execute("""
        CREATE TABLE IF NOT EXISTS plenary_minutes (
            title VARCHAR PRIMARY KEY,
            link VARCHAR,
            sitting INT,
            period INT,
            date DATE,
            description VARCHAR
        )
    """)
    print("Tables created successfully.")

In [74]:
def fill_database(period):
    """
    Fills the database with the scraped data from the bundestag website for a given period.

    Args:
        period (int): The period of the plenary minutes to be saved.

    Returns:
        None
    """
    with open(f'Plenarprotokolle_{period}_wahlperiode.json', 'r', encoding='utf-8') as file:
        documents = json.load(file)

    # Display the data
    for doc in documents:
        try:
            title = doc['title']
            link = doc['link']
            description = doc['description']
            if link == "No link found":
                continue # Skip if no link is found
            else:
                # Extract sitting number
                sitting_match = re.search(r'der (\d+)\. Sitzung', title)
                sitting_number = sitting_match.group(1) if sitting_match else None

                # Extract date
                # 💡 Fix missing space between month and year (e.g., "Mai2022" -> "Mai 2022")
                # This handles all cases like "Mai2022", "Juni2021", etc.
                title = re.sub(r'([a-zäöüßA-ZÄÖÜ]+)(\d{4})', r'\1 \2', title)

                # Extract date in German format (e.g., "dem 15. Mai 2022")
                date_match = re.search(r'dem ([\d\.]+\s\w+\s\d{4})', title)
                german_date_str = date_match.group(1) if date_match else None
                # Convert German date format to datetime object
                date_obj = datetime.strptime(german_date_str, "%d. %B %Y")

                # Convert datetime object to standard format YYYY-MM-DD
                date = date_obj.strftime("%Y-%m-%d")

                # Insert data into the database
                sql = f"""
                    INSERT INTO plenary_minutes (title, link, sitting, period, date, description)
                    VALUES ('{title}', '{link}', {sitting_number}, {period}, '{date}', '{description}')
                """
                con.execute(sql)
        except Exception as e:
            print(f"Error processing document: {doc}")
            print(f"Error message: {e}")

def build_database():
    """
    Build the database with the scraped data from the bundestag website. This is used to save the xml data.

    Returns:
        None
    """
    # Create tables
    create_tables()

    # Fill database with data for each period
    periods = [19,20,21]
    for period in periods:
        fill_database(period)





In [26]:
def download_xml(link):
    """
    Download the XML file from the given link.

    Args:
        link (str): The URL of the XML file to download.

    Returns:
        str: The content of the XML file as a string.
    """

    response = requests.get(link)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to download {link}. Status code: {response.status_code}")
        return None

def save_xml_to_file(xml_content, folder, filename):
    """
    Save the XML content to a file.

    Args:
        xml_content (str): The XML content to save.
        folder (str): The folder where the file will be saved.
        filename (str): The name of the file to save the XML content to.

    Returns:
        None
    """

    if not os.path.exists(folder):
        os.makedirs(folder)

    with open(os.path.join(folder, filename), 'w', encoding='utf-8') as file:
        file.write(xml_content)

def get_xml_files():
    """
    Get the list of XML files from the plenary_minutes table in the database.

    Returns:
        list: A list of tuples containing the link to each XML file.
    """
    periods = [19, 20, 21]  # Define the periods you want to query
    for i in periods:
        df = con.execute(f"SELECT * FROM plenary_minutes WHERE period = {i}").fetchdf()
        for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
            link = row['link']
            filename = f"{i}_{row['sitting']:03}_{row['date'].date()}.xml"
            xml_content = download_xml(link)
            if xml_content:

                save_xml_to_file(xml_content, f"../data/plenary_minutes/wahlperiode_{i}", filename)
            else:
                print(f"Failed to download {filename} for period {i}.")



In [27]:
if __name__ == "__main__":
    #build_database()
    get_xml_files()


Processing rows: 100%|██████████| 6/6 [00:01<00:00,  4.19it/s]


In [8]:
 con.execute("SELECT * FROM plenary_minutes").fetchdf()

,title,link,sitting,period,date,description
0,"Plenarprotokoll der 239. Sitzung von Dienstag,...",https://www.bundestag.de/resource/blob/858472/...,239,19,2021-09-07,XML | 657 KB
1,"Plenarprotokoll der 238. Sitzung von Mittwoch,...",https://www.bundestag.de/resource/blob/855838/...,238,19,2021-08-25,XML | 748 KB
2,"Plenarprotokoll der 237. Sitzung von Freitag, ...",https://www.bundestag.de/resource/blob/850004/...,237,19,2021-06-25,XML | 744 KB
3,Plenarprotokoll der 236. Sitzung von Donnersta...,https://www.bundestag.de/resource/blob/849650/...,236,19,2021-06-24,XML | 2 MB
4,"Plenarprotokoll der 235. Sitzung von Mittwoch,...",https://www.bundestag.de/resource/blob/849430/...,235,19,2021-06-23,XML | 1 MB
...,...,...,...,...,...,...
454,"Plenarprotokoll der 5. Sitzung von Freitag, de...",https://www.bundestag.de/resource/blob/1067754...,5,21,2025-05-16,XML | 438 KB
455,"Plenarprotokoll der 3. Sitzung von Mittwoch, d...",https://www.bundestag.de/resource/blob/1067368...,3,21,2025-05-14,XML | 926 KB
456,"Plenarprotokoll der 4. Sitzung von Donnerstag,...",https://www.bundestag.de/resource/blob/1067464...,4,21,2025-05-15,XML | 988 KB
457,"Plenarprotokoll der 2. Sitzung von Dienstag, d...",https://www.bundestag.de/resource/blob/1064580...,2,21,2025-05-06,XML | 164 KB
